In [2]:
import pandas as pd
import hashlib
import os
from glob import glob
import re

In [3]:
# Change this to your actual triplet folder path
TRIPLETS_FOLDER = "../../Scientific_Novelty_Detection/Triplets/"

# Recursively find all *_triplets.csv files
triplet_files = glob(os.path.join(TRIPLETS_FOLDER, "**", "*_triplets.csv"), recursive=True)

print("Found", len(triplet_files), "triplet files")
for f in triplet_files:
    print(f)

Found 17 triplet files
../../Scientific_Novelty_Detection/Triplets\Blogs\Dia_Blogs_triplets.csv
../../Scientific_Novelty_Detection/Triplets\Blogs\MT_Blogs_triplets.csv
../../Scientific_Novelty_Detection/Triplets\Blogs\QA_Blogs_triplets.csv
../../Scientific_Novelty_Detection/Triplets\Blogs\SA_Blogs_triplets.csv
../../Scientific_Novelty_Detection/Triplets\Blogs\Sum_Blogs_triplets.csv
../../Scientific_Novelty_Detection/Triplets\Novel_Papers\Dia2021_triplets.csv
../../Scientific_Novelty_Detection/Triplets\Novel_Papers\MT2021_triplets.csv
../../Scientific_Novelty_Detection/Triplets\Novel_Papers\QA2021_triplets.csv
../../Scientific_Novelty_Detection/Triplets\Novel_Papers\SA2021_triplets.csv
../../Scientific_Novelty_Detection/Triplets\Novel_Papers\Sum2021_triplets.csv
../../Scientific_Novelty_Detection/Triplets\SKG\Dia_triplets.csv
../../Scientific_Novelty_Detection/Triplets\SKG\MT_triplets.csv
../../Scientific_Novelty_Detection/Triplets\SKG\NLI_triplets.csv
../../Scientific_Novelty_Detection

In [4]:
## Merge all triplets
all_dfs = []

for file in triplet_files:
    df = pd.read_csv(file)

    # Ensure required columns exist
    if "sub" not in df.columns or "obj" not in df.columns:
        raise ValueError(f"Missing 'sub' or 'obj' in {file}")

    all_dfs.append(df[["sub", "obj"]])

triplets_df = pd.concat(all_dfs, ignore_index=True)

print("Total triplet rows:", len(triplets_df))

Total triplet rows: 238088


In [5]:
triplets_df

,sub,obj
0,hyper-parameters,more powerful decoder
1,more powerful decoder,higher conversational quality
2,ablation analysis,hyper-parameters
3,tuned decoding,ssa score
4,ssa score,79 %
...,...,...
238083,results,proposed model
238084,our model,longer sequences
238085,longer sequences,introduction and the sentences
238086,introduction and the sentences,extractor


In [6]:
## Normalise Entity Strings
def normalize_entity(text):
    if pd.isna(text):
        return None

    text = str(text).strip().lower()

    # Remove excessive quotes
    text = text.replace('"""', '"')
    text = text.replace("''", "'")

    # Remove extra spaces
    text = " ".join(text.split())

    return text

triplets_df["sub"] = triplets_df["sub"].apply(normalize_entity)
triplets_df["obj"] = triplets_df["obj"].apply(normalize_entity)

In [7]:
raw_entities = set(triplets_df["sub"].dropna()) | set(triplets_df["obj"].dropna())
print("Raw unique entities:", len(raw_entities))

Raw unique entities: 119379


In [8]:
def is_valid_entity(text):
    if not text:
        return False

    text = text.strip()

    # Minimum length
    if len(text) < 3:
        return False

    # Remove parenthesis-dominated patterns
    if text.startswith("("):
        return False

    if re.fullmatch(r"\(.*\)", text):
        return False

    # Remove set notation / math blocks
    if text.startswith("{") or text.startswith("|"):
        return False

    if re.search(r"\{.*\}", text):
        return False

    # Remove scientific notation
    if re.search(r"\d+e[-\s]?\d+", text):
        return False

    # Remove heavy numeric ratio
    digit_ratio = sum(c.isdigit() for c in text) / len(text)
    if digit_ratio > 0.4:
        return False

    # Remove hardware/config patterns
    if "gpu" in text:
        return False

    if "units" in text:
        return False

    if "encoder" in text and "decoder" in text:
        return False

    # Remove statistical fragments
    if text.startswith("~"):
        return False

    if "%" in text:
        return False

    stat_patterns = [
        " more ",
        " lower",
        " lines",
        " questions",
        " entities",
        " pp",
        " ratio"
    ]

    for p in stat_patterns:
        if p in text:
            return False

    if re.search(r"\d+\s?[km]", text):
        return False

    if re.search(r"\d+\.?\d*x", text):
        return False

    # Must contain alphabet
    if not re.search(r"[a-zA-Z]", text):
        return False

    # Remove pure punctuation
    if re.fullmatch(r"[\W_]+", text):
        return False

    return True

In [9]:
## Extract unique Entities
clean_entities = {e for e in raw_entities if is_valid_entity(e)}
print("Clean unique entities:", len(clean_entities))

Clean unique entities: 108816


In [10]:
## Assign entity IDs
def generate_entity_id(entity_string):
    return "E_" + hashlib.md5(entity_string.encode("utf-8")).hexdigest()[:10]

entity_data = []

for entity in sorted(clean_entities):
    entity_id = generate_entity_id(entity)
    entity_data.append({
        "node_id": entity_id,
        "node_type": "Entity",
        "name": entity
    })

entity_nodes_df = pd.DataFrame(entity_data)

In [11]:
## VERIFY NO HASH COLLISIONS
print("Duplicate entity IDs:",
      entity_nodes_df["node_id"].duplicated().sum())

assert entity_nodes_df["node_id"].is_unique

Duplicate entity IDs: 0


In [17]:
assert entity_nodes_df["name"].isnull().sum() == 0

In [12]:
entity_nodes_df.to_csv("../outputs/entity_nodes.csv", index=False)

print("entity_nodes.csv created successfully.")

entity_nodes.csv created successfully.


In [13]:
entity_nodes_df.sample(50)

,node_id,node_type,name
20086,E_83c45eb71b,Entity,chinese- to - english task
13207,E_940ff42ba7,Entity,average of the embedding vectors
35316,E_1d90d19da5,Entity,existing top performance model
95403,E_7f1323c006,Entity,system pastr
17481,E_d79b581f19,Entity,bleu and gender accuracy
23860,E_92d11b339c,Entity,conversational kb - qa agent
60371,E_09d4ca34b1,Entity,multi-hop qg
3024,E_7db35e539a,Entity,2.12 bleu points
48530,E_42744c1f33,Entity,joint bpe vocabularies
29538,E_2bcd7c0a47,Entity,document rotation


In [18]:
print("Total entities:", len(entity_nodes_df))
print("Duplicate names:",
      entity_nodes_df["name"].duplicated().sum())
print("Duplicate IDs:",
      entity_nodes_df["node_id"].duplicated().sum())

print("Entity node checks passed.")

Total entities: 108816
Duplicate names: 0
Duplicate IDs: 0
Entity node checks passed.
